In [150]:
# You should be able to just run the whole script to align
# Just run all cells
#this
import os
import yaml
import time
import numpy as np
import pandas as pd
import shelve
import sys
import random
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from playsound import playsound

from pathlib import Path

sys.path.append('../../')
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice
# from geecs_python_api.controls.experiment.experiment import Experiment



In [151]:
#this

# This cell for BELLA LPMode setup
GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Bella")
camera = GeecsDevice('CAM-PL1-LPMode')
camera.subscribe_var_values(['Target.X', 'Target.Y', 'Target2.X', 'Target2.Y'])
exposure = camera.get('exposure')
print(f"Camera exposure is {exposure}")
print(camera.state)

picos = GeecsDevice('MCD-PL1-picosOFI1')
picos.subscribe_var_values(['Holy mirror horizontal', 'Holy mirror vertical'])
print("Initial pico motor positions:")
print(f"1: {picos.get('Position.Axis 3')}") # holy mirror horizontal
print(f"2: {picos.get('Position.Axis 4')}") # holy mirror vertical
print(picos.state)

hexapod = GeecsDevice('HEX-PL1-2')
hexapod.subscribe_var_values(['ypos', 'zpos']) # ypos for x, zpos for y
print("Initial hexapod positions:")
print(f"ypos: {hexapod.get('ypos')}") # axicon horizontal
print(f"zpos: {hexapod.get('zpos')}") # axicon vertical
print(hexapod.state)

stage = GeecsDevice('STAGE-PL1-LPmodeLong')
stage.subscribe_var_values(['position'])
print("Initial stage positions:")
print(f"ypos: {stage.get('position')}") # stage pos
print(stage.state)

Camera exposure is 0.005
{'fresh': False, 'shot number': None, 'GEECS device error': False, 'Device alive on instantiation': True, 'exposure': 0.005}
Initial pico motor positions:
1: 0.0
2: 0.0
{'fresh': True, 'shot number': 0, 'GEECS device error': False, 'Device alive on instantiation': True, 'Holy mirror horizontal': 0.0, 'last_message': 'MCD-PL1-picosOFI1>>0>>Device Status nval,Initialized nvar,\r\ndevice error nval, nvar,\r\ndevice preset nval, nvar,\r\n', 'Device Status': 'Initialized', '': '', 'device error': '', 'device preset': '', 'Holy mirror vertical': 0.0}
Initial hexapod positions:
ypos: 0.57
zpos: -2.46
{'fresh': True, 'shot number': 0, 'GEECS device error': False, 'Device alive on instantiation': True, 'last_message': 'HEX-PL1-2>>0>>Device Status nval,Initialized nvar,\r\ndevice error nval, nvar,\r\ndevice preset nval,Amp3 45 J, Megaattenuator nvar,\r\nypos nval,0.570000 nvar,\r\nzpos nval,-2.460000 nvar,\r\n', 'Device Status': 'Initialized', '': '', 'device error': '',

In [152]:
steps_per_pixel_picos_x = 5.5555 # at stage pos 570
# at 550: -5.1546
# ratio: 1.07777519109
# This means stage pos 570 is 257.15140933mm away from axis of rotation
steps_per_pixel_picos_y = 5.6818
steps_per_pixel_conex_x = -0.0001201923
steps_per_pixel_conex_y = 0.00016286644
steps_per_pixel_hex_x = 6.5e-4
steps_per_pixel_hex_y = -6.5e-4

mcd_dict = {
    'HM_x': { # Holy Mirror pico
        'device': picos, 
        'name': 'Position.Axis 3', 
        'alias': 'Holy mirror horizontal', 
        'calib': steps_per_pixel_picos_x, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': picos.get('Position.Axis 3') - 100000, # Software limits for beampointing
        'max': picos.get('Position.Axis 3') + 100000, # You can change the +- to be larger if the beam doesn't fit
       },
    'HM_y': { # Holy mirror pico
        'device': picos, 
        'name': 'Position.Axis 4', 
        'alias': 'Holy mirror vertical', 
        'calib': steps_per_pixel_picos_y, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'y', 
        'min': picos.get('Position.Axis 4') - 100000,
        'max': picos.get('Position.Axis 4') + 100000,
       },
    'axicon_x': {
        'device': hexapod, 
        'name': 'ypos', 
        'alias': 'ypos', 
        'calib': steps_per_pixel_hex_x, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': hexapod.get('ypos') - 0.4,
        'max': hexapod.get('ypos') + 0.4,
    },
    'axicon_y': {
        'device': hexapod, 
        'name': 'zpos', 
        'alias': 'zpos', 
        'calib': steps_per_pixel_hex_y, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'y',
        'min': hexapod.get('zpos') - 0.4,
        'max': hexapod.get('zpos') + 0.4,
    },
}

In [153]:
#this
# Hyperparameters
SHOTS_TO_AVG = 3
TIME_BETWEEN_SHOTS = 0.1 # seconds

In [154]:
def set_with_limit(mcd, value, min_val=None, max_val=None):
    """
    Set a parameter with software limits, default is no limit
    This is a blocking function since we are setting
    
    mcd: device dictionary in mcd_dict
    value: value to set to
    min_val, max_val: you can temporarily override the software limits by specifying
    a non-null value
    
    return: actual value set
    """
    if not min_val:
        min_val = mcd['min']
    if not max_val:
        max_val = mcd['max']
        
    if min_val > max_val:
        raise Exception('min_val must be smaller than max_val')
        
    mcd['device'].set(mcd['name'], max(min_val, min(max_val, value)))
    return max(min_val, min(max_val, value))

In [155]:
#this
# Beam pointing onto target
def obj_func(targetx, targety, beamx, beamy):
    return (targetx - beamx)**2 + (targety - beamy)**2

def move_random(pico_num, picos, dmin, dmax):
    """
    pico_num: an index for mcd_dict
    picos: GEECS picos object
    moves the pico between [dmin, dmax] randomly from its original position
    """
    picomotor = mcd_dict[pico_num]
    new_pos = picos.state[picomotor['name']] + random.randint(dmin, dmax)
    print(f"moving motor {pico_num} to position {new_pos}")
    picos.set(picomotor['name'], int(picos.state[picomotor['name']] + random.randint(dmin, dmax)))
    return

def get_centroid(camera, num_shots, time_between_shots):
    """
    Returns (x,y) of centroid of beam
    """
    centroidx = []
    centroidy = []
    # Average over SHOTS_TO_AVG shots
    for i in range(num_shots):
        # While loops since sometimes the camera doesn't send back a value for centroid
        centroidx1, centroidy1 = None, None
        while not isinstance(centroidx1, float):
            centroidx1 = camera.get('Centroidx')
#             print(centroidx1)
        centroidx.append(centroidx1)
        while not isinstance(centroidy1, float):
            centroidy1 = camera.get('Centroidy')
        centroidy.append(centroidy1)
        print(f"Centroid is {centroidx1}, {centroidy1}")
        time.sleep(time_between_shots)

    centroidx = np.mean(np.array(centroidx))
    centroidy = np.mean(np.array(centroidy))
    return centroidx, centroidy
    
    

In [156]:
def move_pico_to_pixel(camera, obj_target, opt_rate, pico_x, pico_y, target_pix_x, target_pix_y):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing when we are less than obj_target pixels in the given axis
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico_x, pico_y: object in the mcd dict, this should be a picomotor
    target_pix_x, target_pix_y: pixel to move to
    
    returns: iterations, obj_func()
    """
    
    # Helper function to get target position
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        ob = (target_pix_x - centroidx)**2 + (target_pix_y - centroidy)**2
        print(f"In move pico: Objective is {ob}")
        return ob, centroidx, centroidy
        
    print(f"Target pixel is: (x,y) = ({target_pix_x}, {target_pix_y})")
    # Move mcds until beam is close enough to target
    iterations = 0
    ob, centroidx, centroidy = obj()
    
    while ob > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        pico_pos_x = pico_x['device'].state[pico_x['alias']]
        pico_pos_y = pico_y['device'].state[pico_y['alias']]
        print(f"Original pico position is ({pico_pos_x}, {pico_pos_y})")
        
        dx = target_pix_x - centroidx
        dy = target_pix_y - centroidy
        print(f"original centroid position = ({centroidx}, {centroidy})")
        print(f"dx = {dx}, dy = {dy}")
        move_x = dx * pico_x['calib'] * opt_rate
        move_y = dy * pico_y['calib'] * opt_rate
        set_with_limit(pico_x, int(pico_pos_x + move_x))
        set_with_limit(pico_y, int(pico_pos_y + move_y))
        
        print(f"Final position is picox = ({pico_x['device'].state[pico_x['alias']]}), "
              f"picoy = ({pico_y['device'].state[pico_y['alias']]})")
        time.sleep(1)
        
        ob, centroidx, centroidy = obj()
        
    
    return iterations, ob

def move_axicon_to_pixel(camera, obj_target, opt_rate, axicon_x, axicon_y, target_pix_x, target_pix_y):
    """
    Moves a beam onto a target with axicon1 and axicon2
    
    camera: GEECS camera object
    obj_target: stop optimizing when we are less than obj_target pixels in the given axis
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    axicon_x, axicon_y: object in the mcd dict, this should be a axicon direction
    target_pix_x, target_pix_y: pixel to move to
    
    returns: iterations, obj_func()
    """
    
    # Helper function to get target position
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        ob = (target_pix_x - centroidx)**2 + (target_pix_y - centroidy)**2
        print(f"In move axicon: Objective is {ob}")
        return ob, centroidx, centroidy
        
    print(f"Target pixel is: (x,y) = ({target_pix_x}, {target_pix_y})")
    # Move mcds until beam is close enough to target
    iterations = 0
    ob, centroidx, centroidy = obj()
    
    while ob > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        axicon_pos_x = axicon_x['device'].state[axicon_x['alias']]
        axicon_pos_y = axicon_y['device'].state[axicon_y['alias']]
        print(f"Original axicon position is ({axicon_pos_x}, {axicon_pos_y})")
        
        dx = target_pix_x - centroidx
        dy = target_pix_y - centroidy
        print(f"original centroid position = ({centroidx}, {centroidy})")
        print(f"dx = {dx}, dy = {dy}")
        move_x = dx * axicon_x['calib'] * opt_rate
        move_y = dy * axicon_y['calib'] * opt_rate
        set_with_limit(axicon_x, axicon_pos_x + move_x)
        set_with_limit(axicon_y, axicon_pos_y + move_y)
        
        print(f"Final position is axiconx = ({axicon_x['device'].state[axicon_x['alias']]}), "
              f"axicony = ({axicon_y['device'].state[axicon_y['alias']]})")
        time.sleep(1)
        
        ob, centroidx, centroidy = obj()
    
    return iterations, ob
    

In [157]:
def align_2_targets_at_once(mcd_1x, mcd_1y, mcd_2x, mcd_2y, camera, obj_target, 
                            opt_rate, stage_pos_a, stage_pos_b, lambda_1, lambda_2):
    """
    Uses 2 different calibrations to align to 2 targets at once
    When using this function, make sure that "Target.X" and "Target.Y" are the upstream camera
    position and "Target2.X"/"Target2.Y" are the downstream position
    
    mcd_1x, mcd_1y: dictionary entries for upstream mcd x and y controllers
    mcd_2x, mcd_2y: dictionary entries for downstream mcd x and y controllers
    It is assumed that the upstream mcd "aligns to" the closer position, while the downstream mcd
    "aligns to" the further position in traditional alignment
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    stage_pos_a: closer numerical value for stage position
    stage_pos_b: far numerical value for stage position
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    lambda_1: ratio of pixels moved of stage_pos_b/stage_pos_a for mcd_1
    lambda_2: ratio of pixels moved of stage_pos_b/stage_pos_a for mcd_2
    """
    #Delta x_a = move_1a + move_2a
    #Delta x_b = lambda_1*move_1a + lambda_2*move_2a
    #move_1a means the number of pixels to move on mcd 1
    begin_time = time.time()
    
    t1 = time.time()
    stage.set('position', stage_pos_a)
    print(f"Time for stage to move: {time.time() - t1}")
    time.sleep(5)
    print(f"Time after sleep: {time.time() - t1}")
    centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    t1 = time.time()
    stage.set('position', stage_pos_b)
    print(f"Time for stage to move: {time.time() - t1}")
    time.sleep(5)
    print(f"Time after sleep: {time.time() - t1}")
    centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    
    while True:
        # X matrix equation (A * \vec{pos} = \vec{b})
        # A is matrix of calibrations
        # pos = positions/values of mcds
        # b = pixel difference in target vs centroid
        # Must do this for x and y direction

        # x-dir
        A = np.array([
            [1, 1],
            [lambda_1, lambda_2],
        ])
#         print("Matrix A is (X-dir):", A)
        b = np.array([
            [camera.state['Target.X'] - centroid_close_x],
            [camera.state['Target2.X'] - centroid_far_x],
        ])
        pixel_moves_x = np.matmul(np.linalg.inv(A), b).flatten()
        pixel_moves_x *= [lambda_1, lambda_2]
        
        # y-dir
        A = np.array([
            [1, 1],
            [lambda_1, lambda_2],
        ])
#         print("Matrix A is (X-dir):", A)
        b = np.array([
            [camera.state['Target.Y'] - centroid_close_y],
            [camera.state['Target2.Y'] - centroid_far_y],
        ])
        pixel_moves_y = np.matmul(np.linalg.inv(A), b).flatten()
        pixel_moves_y *= [lambda_1, lambda_2]
        
#         print(f"b = {b}")
        print(f"pixel moves x: {pixel_moves_x}")
        print(f"pixel moves y: {pixel_moves_y}")
        stage.set('position', stage_pos_b)
        # move 
        move_axicon_to_pixel(camera, obj_target**2, 1, mcd_1x, mcd_1y, 
                             centroid_far_x + pixel_moves_x[0],
                             centroid_far_y + pixel_moves_y[0])
        move_pico_to_pixel(camera, obj_target**2/2, opt_rate, mcd_2x, mcd_2y, 
                           centroid_far_x + pixel_moves_x[0] + pixel_moves_x[1], 
                           centroid_far_y + pixel_moves_y[0] + pixel_moves_y[1])
        
        # Check to see if moved to correct positions
        t1 = time.time()
        stage.set('position', stage_pos_a)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(5)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        t1 = time.time()
        stage.set('position', stage_pos_b)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(5)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        
        obj1 = np.abs(centroid_close_x - camera.state['Target.X'])
        obj2 = np.abs(centroid_far_x - camera.state['Target2.X'])
        
        print(f"\nObj 1 = {obj1}, Obj2 = {obj2}\n")
        
        if (obj1 < obj_target) and (obj2 < obj_target):
            break
            
    print("Close delta x:", centroid_close_x - camera.state['Target.X'])
    print("Close delta y:", centroid_close_y - camera.state['Target.Y'])
    print("Far delta x:", centroid_far_x - camera.state['Target2.X'])
    print("Far delta y:", centroid_far_y - camera.state['Target2.Y'])
    print(f"Alignment took {time.time() - begin_time} seconds")
    
    return
    
    
    

In [158]:
stage_pos_upstream = 590
stage_pos_downstream = 570
ratio = (827 - stage_pos_downstream) / (827 - stage_pos_upstream)
assert(ratio > 1)

In [159]:
print(ratio)

1.0843881856540085


In [160]:
align_2_targets_at_once(mcd_dict['axicon_x'], mcd_dict['axicon_y'], mcd_dict['HM_x'], mcd_dict['HM_y'], camera, 3, 
                            0.8, stage_pos_upstream, stage_pos_downstream, 1, ratio)

Time for stage to move: 1.3065097332000732
Time after sleep: 6.308150053024292
Centroid is 983.713928, 170.390656
Centroid is 986.276855, 172.250031
Centroid is 984.728027, 172.310455
Time for stage to move: 1.2696082592010498
Time after sleep: 6.2712485790252686
Centroid is 1143.637085, 350.989166
Centroid is 1142.836182, 347.380768
Centroid is 1142.99292, 347.673676
pixel moves x: [ 572.49586915 -478.65126482]
pixel moves y: [ 632.16486793 -475.84607127]
Target pixel is: (x,y) = (1715.6512648166668, 980.8460712666667)
Centroid is 1140.737549, 347.101532
Centroid is 1140.451538, 347.594788
Centroid is 1143.242554, 346.851379
In move axicon: Objective is 731205.2784983995
Iteration 1
Original axicon position is (0.57, -2.46)
original centroid position = (1141.4772136666666, 347.18256633333334)
dx = 574.1740511500002, dy = 633.6635049333333
Final position is axiconx = (0.9432), axicony = (-2.86)
Centroid is 1865.6875, 1062.447144
Centroid is 1861.101929, 1062.572632
Centroid is 1861.930

In [161]:
camera.close()
picos.close()
hexapod.close()
stage.close()

[TcpSubscriber class, method "async_listener"] 'NoneType' object has no attribute 'recv'
[TcpSubscriber class, method "async_listener"] [WinError 10038] An operation was attempted on something that is not a socket
[TcpSubscriber class, method "async_listener"] [WinError 10038] An operation was attempted on something that is not a socket
[TcpSubscriber class, method "async_listener"] [WinError 10038] An operation was attempted on something that is not a socket


In [162]:
audio = r"C:\GEECS\Developers Version\source\GEECS-Plugins\GEECS-PythonAPI\PW_scripts\Raymond\applause.mp3"
playsound(audio)

KeyboardInterrupt: 